In [3]:
!pip install httpimport

In [4]:
import pandas as pd 

import boto3 
import sagemaker 
import httpimport 
import io 

from pyathena import connect 

In [7]:
# initialize sagemaker sesssion and env var
sess = sagemaker.Session() 
bucket = sess.default_bucket() 
role = sagemaker.get_execution_role() 
region = boto3.Session().region_name
sm = boto3.client("sagemaker", region_name=region) 

In [8]:
# set s3 variables
# s3 = boto3.client('s3')
# bucket_name = 'jc-bucket-ads508-2025-finalproject'

# file_benefits = './employee_benefits_2010_2024.csv'
# s3_key_benefits = 'datasets/employee_benefits_2010_2024.csv'
s3_public_path = "s3://usd-team1-ads508/ld_data/"
s3_private_path = f"s3://{bucket}/ld_data"

print(f"Public path: {s3_public_path}")

Public path: s3://usd-team1-ads508/ld_data/


In [9]:
# import jcds 
with httpimport.github_repo('junclemente', 'jcds', ref='0.1.0'):
    import jcds.eda as jeda

In [ ]:
# load dataset from s3 into dataframe
# obj = s3.get_object(Bucket=bucket_name, Key=s3_key_benefits)
# # read Excel file into dataframe
# df = pd.read_csv(io.BytesIO(obj['Body'].read()))

# print(df.head())

In [10]:
# store paths
%store s3_public_path
%store s3_private_path

Stored 's3_public_path' (str)
Stored 's3_private_path' (str)


In [11]:
# copy csv files from public to private bucket
!aws s3 cp --recursive {s3_public_path} {s3_private_path}/ --include "*.csv" > /dev/null 2>&1 

In [14]:
# list objects in bucket
s3 = boto3.client("s3") 
bucket_name = "usd-team1-ads508"
response = s3.list_objects_v2(Bucket=bucket_name)

# extract file details
if "Contents" in response:
    file_list = [{"Key": obj["Key"], "Size (KB)": obj["Size"] / 1024, "Last Modified": obj["Last Modified"]} for obj in response["Contents"]]

    # convert to pandas df
    df_files = pd.DataFrame(file_list) 

    # display 
    print(df_files) 
else: 
    print("No files founds in the S3 bucket.")

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied